<a href="https://colab.research.google.com/github/agentkyo/Google_App_Script/blob/main/Automa%C3%A7%C3%A3o_Wordpress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv gspread google-auth --quiet

import requests
import os
import base64
from dotenv import load_dotenv, find_dotenv
import gspread
from google.colab import auth
from google.auth import default
from google.colab import userdata

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

load_dotenv(find_dotenv())

class ShipayWordpress:
    def __init__(self):
        self.site_url = userdata.get("SITE_URL")
        self.username = userdata.get("WORDPRESS_USERNAME")
        self.password = userdata.get("WORDPRESS_API_KEY")

    def authenticate(self):
        token = base64.b64encode(f"{self.username}:{self.password}".encode()).decode()
        headers = {"Content-Type": "application/json", "Authorization": f"Basic {token}"}
        response = requests.get(f"{self.site_url}/wp-json/", headers=headers)
        if response.status_code == 200:
            print("Authenticated")
            return response.json()
        else:
            print(f"Failed to authenticate: {response.status_code}")
            return None

    def get_wordpress_posts(self):
        token = base64.b64encode(f"{self.username}:{self.password}".encode()).decode()
        headers = {"Content-Type": "application/json", "Authorization": f"Basic {token}"}
        endpoint = f"{self.site_url}/wp-json/wp/v2/posts"
        params = {"per_page": 100, "page": 1}
        response = requests.get(endpoint, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to retrieve posts on page 1: {response.status_code}")
            return None
        all_posts = response.json()
        total_pages = int(response.headers.get("X-WP-TotalPages", 1))
        for page in range(2, total_pages + 1):
            params["page"] = page
            response = requests.get(endpoint, headers=headers, params=params)
            if response.status_code != 200:
                print(f"Failed to retrieve posts on page {page}: {response.status_code}")
                break
            all_posts.extend(response.json())
        return all_posts

def format_posts_for_sheet(posts):
    data = [["ID", "Título", "Conteúdo", "Data"]]
    for post in posts:
        post_id = post.get("id")
        title = post.get("title", {}).get("rendered", "")
        content = post.get("content", {}).get("rendered", "")
        date = post.get("date")
        data.append([post_id, title, content, date])
    return data

def update_google_sheet(sheet_id, worksheet_name, data):
    sh = gc.open_by_key(sheet_id)
    try:
        worksheet = sh.worksheet(worksheet_name)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sh.add_worksheet(title=worksheet_name, rows="100", cols="20")
    worksheet.clear()
    worksheet.update(range_name='A1', values=data)
    print("Planilha atualizada com sucesso!")

wp_client = ShipayWordpress()
posts = wp_client.get_wordpress_posts()
if posts is None:
    print("Não foi possível recuperar os posts.")
else:
    table_data = format_posts_for_sheet(posts)
    SHEET_ID = "1r1GLZscR40SvwwpEWSzfS3FNZNBjoRwn-br1LJjBFDc"
    WORKSHEET_NAME = "wordpress"
    update_google_sheet(SHEET_ID, WORKSHEET_NAME, table_data)


Planilha atualizada com sucesso!
